<a href="https://colab.research.google.com/github/shivamsinghtomar78/ML-Projects-/blob/main/Fake_news_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the DataSet:

1.Id:Unique id for a news article

2.title:the title of a news article

3.author: author of the news article

4.text: the text of the article ;could be compltete

5.label: a label that marks whether the news article is real or fake

     1: Fake news

      0: real news

In [30]:
import numpy as np
import pandas as pd
import re
import sklearn
import pickle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
#printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [33]:
#Loading the dataset to a pandas Dataframes
news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='skip')

In [34]:
news_dataset.shape

(20800, 5)

In [35]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [36]:
#counting the number of missing values in the dataset
print("\nMissing values:")
print(news_dataset.isnull().sum())


Missing values:
id           0
title      558
author    1957
text        39
label        0
dtype: int64


In [37]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [38]:
#Merging the author name and news title
news_dataset['content'] = news_dataset['title'] + ' ' + news_dataset['text']
print("\nCombined content:")
print(news_dataset['content'])


Combined content:
0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Why the Trut...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799    What Keeps the F-35 Alive   David Swanson is a...
Name: content, Length: 20800, dtype: object


In [39]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

Stemming:

Stemming is the process of reducing a word to its root word

example:

actor,actress,acting-->act

In [40]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [41]:
# Apply stemming to content
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [43]:
# Separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values
print("\nFeature array:")
print(X)
print("\nLabel array:")
print(Y)
print("\nLabel shape:", Y.shape)


Feature array:
['hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light un

In [44]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

Spliting the dataSet to trainig and test data

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

Training the model Logistic Regression

In [46]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

 Accuracy Score

In [47]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data:', training_data_accuracy)

Accuracy score of the training data: 0.9742788461538462


In [48]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data:', test_data_accuracy)

Accuracy score of the test data: 0.948076923076923


In [52]:
existing_dataset = pd.read_csv('/content/train.csv', on_bad_lines='skip')

In [57]:
# Load the new datasets
fake_dataset = pd.read_csv('/content/Fake.csv', on_bad_lines='skip')
true_dataset = pd.read_csv('/content/True.csv', on_bad_lines='skip')

In [58]:
# Add labels to the new datasets
fake_dataset['label'] = 1  # 1 for Fake news
true_dataset['label'] = 0  # 0 for Real news

In [59]:
# Combine the new datasets
new_dataset = pd.concat([fake_dataset, true_dataset], ignore_index=True)

In [60]:
# Combine the existing dataset with the new dataset
combined_dataset = pd.concat([existing_dataset, new_dataset], ignore_index=True)

In [61]:
# Replace null values with empty string
combined_dataset = combined_dataset.fillna('')

In [62]:
# Merge the title and text columns
combined_dataset['content'] = combined_dataset['title'] + ' ' + combined_dataset['text']

In [63]:
# Stemming function
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [64]:
# Apply stemming to content
combined_dataset['content'] = combined_dataset['content'].apply(stemming)

In [65]:
# Separate the data and label
X = combined_dataset['content'].values
Y = combined_dataset['label'].values

In [66]:
# Convert textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [67]:
# Split the dataset into training and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [68]:
# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [69]:
# Evaluate the model
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data:', training_data_accuracy)

Accuracy score of the training data: 0.9710415160394231


In [70]:

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data:', test_data_accuracy)

Accuracy score of the test data: 0.9545662100456621


In [71]:
# Save the updated model and vectorizer
model_data = {
    'model': model,
    'vectorizer': vectorizer
}
with open('updated_trained_model.pkl', 'wb') as file:
    pickle.dump(model_data, file)

print("Updated model and vectorizer saved successfully!")

Updated model and vectorizer saved successfully!
